In [75]:
import requests, json, math

In [34]:
url = 'https://search-coca-ngram-yn3fvpdx4cm7gfvfb6ln7xcgea.us-east-2.es.amazonaws.com/coca-v1/_search/'

In [178]:
def getNormalizedFreq(s):
    r = requests.post(url, headers = {'Content-Type':'application/json'},
                  data = json.dumps({
                    "_source": ["count","gram"],
                    "query": {
                        "query_string": {
                            "query": "name:\""+s+"\""
                        }
                    }}))
    hits = r.json()['hits']['hits']
    
    if len(hits)>0:
        cnt = hits[0]['_source']['count']
        s_len = hits[0]['_source']['gram']
        return math.pow(s_len,s_len)*cnt
    
    return 0

In [179]:
def returnBoundaries(s):
    #ret = []
    for i in range(pow(2,(len(s)-1))):
        f="{0:0"+str(len(s)-1)+"b}"
        sArr = [0]
        cnt = 0
        for x in f.format(i):
            cnt +=1
            if x=='1':
                sArr.append(cnt)
        sArr.append(len(s))    
        rArr=[]
        for i in range(len(sArr)-1):
            rArr.append(' '.join(s[sArr[i]:sArr[i+1]]))
        #ret.append(rArr)
        yield rArr
    #return ret

In [203]:
s = 'new york times square dance'

def getMaxScoreSegment(s):
    weight = {}
    segmentScores = []
    for segments in returnBoundaries(s.split()):
        score = 0
        for word in segments:
            if len(word.split())>=2:
                if word not in weight:
                    weight[word] = getNormalizedFreq(word) 
                if weight[word]==0:
                    score = -1
                    break
                score += weight[word]
        segmentScores.append((segments,score) ) 
    #return sorted(segmentScores,key=lambda k:k[1],reverse=True)
    return max(segmentScores,key=lambda k:k[1])

In [204]:
getMaxScoreSegment(s)

(['new york', 'times square', 'dance'], 590588.0)

In [ ]:
'new york times square dance'

'new |york times square dance'
'new york |times square dance'
'new york times |square dance'
'new york times square |dance'

'new |york |times square dance'
'new |york times |square dance'
'new |york times square |dance'
'new york |times| square dance'
'new york |times square| dance'
'new york times| square| dance'